In [5]:
import xarray as xr
import pandas as pd

In [ ]:
#GOALS: 
#Correlation matrix between the ngpus and the norksy800
#Spectral analysis between the ngpus and the norkyst800
#plot f/h as contourlines with u & v
ds = xr.open_dataset(f'/lustre/storeB/project/fou/hi/foccus/experiments/ngpus-2017-24/inference/lam-48h-last.nc')

In [9]:
ds

<xarray.Dataset> Size: 5GB
Dimensions:            (time: 17, values: 3076056)
Coordinates:
  * time               (time) datetime64[ns] 136B 2024-04-02 ... 2024-04-04
Dimensions without coordinates: values
Data variables: (12/27)
    latitude           (values) float32 12MB ...
    longitude          (values) float32 12MB ...
    h                  (time, values) float32 209MB ...
    salinity_0         (time, values) float32 209MB ...
    sea_mask           (time, values) float32 209MB ...
    temperature_0      (time, values) float32 209MB ...
    ...                 ...
    rain               (time, values) float32 209MB ...
    river_binary_mask  (time, values) float32 209MB ...
    sin_julian_day     (time, values) float32 209MB ...
    sin_latitude       (time, values) float32 209MB ...
    sin_local_time     (time, values) float32 209MB ...
    sin_longitude      (time, values) float32 209MB ...